<a href="https://colab.research.google.com/github/Namanm23/Deep-Learning-Codes/blob/main/Adversarial_Generative_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from tqdm import tqdm  # for progress bar

# Load and preprocess MNIST dataset
(x_train, _), (_, _) = mnist.load_data()
x_train = (x_train.astype(np.float32) - 127.5) / 127.5  # Normalize to [-1, 1]
x_train = np.expand_dims(x_train, axis=-1)

# Dimensions
img_rows, img_cols, channels = x_train.shape[1:]
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

# Build generator
def build_generator():
    model = Sequential([
        Input(shape=(latent_dim,)),
        Dense(256),
        LeakyReLU(negative_slope=0.2),
        Dense(512),
        LeakyReLU(negative_slope=0.2),
        Dense(1024),
        LeakyReLU(negative_slope=0.2),
        Dense(np.prod(img_shape), activation='tanh'),
        Reshape(img_shape)
    ])
    return model

# Build discriminator
def build_discriminator():
    model = Sequential([
        Input(shape=img_shape),
        Flatten(),
        Dense(512),
        LeakyReLU(negative_slope=0.2),
        Dense(256),
        LeakyReLU(negative_slope=0.2),
        Dense(1, activation='sigmoid')
    ])
    return model

# Instantiate models
optimizer = Adam(0.0002, 0.5)
generator = build_generator()
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Build combined GAN model
z = Input(shape=(latent_dim,))
img = generator(z)
discriminator.trainable = False  # Freeze discriminator in combined model
validity = discriminator(img)
combined = Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

# Save images function
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5  # Rescale to [0, 1]

    fig, axs = plt.subplots(r, c, figsize=(5,5))
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(f"mnist_epoch_{epoch}.png")
    plt.close()

# Training function
def train(epochs=1000, batch_size=64, save_interval=200):
    half_batch = batch_size // 2

    for epoch in tqdm(range(epochs), desc="Training Progress"):
        # Train discriminator
        idx = np.random.randint(0, x_train.shape[0], half_batch)
        real_imgs = x_train[idx]

        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        fake_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_imgs, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_labels = np.ones((batch_size, 1))
        g_loss = combined.train_on_batch(noise, valid_labels)

        # Print progress
        if epoch % 100 == 0 or epoch == epochs - 1:
            print(f"{epoch} [D loss: {d_loss[0]:.4f}, acc.: {100*d_loss[1]:.2f}%] [G loss: {g_loss:.4f}]")

        if epoch % save_interval == 0 or epoch == epochs - 1:
            save_imgs(epoch)

# Start training
train(epochs=1000, batch_size=64, save_interval=250)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Training Progress:   0%|          | 0/1000 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 808ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


0 [D loss: 1.2093, acc.: 22.66%] [G loss: 0.7649]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step


...

Training Progress: 100%|█████████▉| 998/1000 [01:27<00:00,  7.60it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


Training Progress: 100%|█████████▉| 999/1000 [01:27<00:00,  7.60it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
999 [D loss: 5.8600, acc.: 0.30%] [G loss: 0.0026]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step


Training Progress: 100%|██████████| 1000/1000 [01:27<00:00, 11.39it/s]
